<a href="https://colab.research.google.com/github/kinjaljoshi/s2t-training/blob/main/testing_after_react_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect to storage bucket
from google.colab import auth
auth.authenticate_user()


In [ ]:
PROJECT_ID = 'llm-text-to-sql-445914'
!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
!gsutil -m cp -r gs://llm-pipeline-resources/models/fine_tuned_model_v1.1 /content/

Copying gs://llm-pipeline-resources/models/fine_tuned_model_v1.1/README.md...
Copying gs://llm-pipeline-resources/models/fine_tuned_model_v1.1/adapter_config.json...
Copying gs://llm-pipeline-resources/models/fine_tuned_model_v1.1/adapter_model.safetensors...
\ [3/3 files][ 18.0 MiB/ 18.0 MiB] 100% Done                                    
Operation completed over 3 objects/18.0 MiB.                                     


In [ ]:
!ls -l /content/fine_tuned_model

total 3096216
-rw-r--r-- 1 root root        709 Jan  3 00:38 adapter_config.json
-rw-r--r-- 1 root root   18915328 Jan  3 00:38 adapter_model.safetensors
-rw-r--r-- 1 root root        816 Jan  3 00:38 config.json
-rw-r--r-- 1 root root        142 Jan  3 00:38 generation_config.json
-rw-r--r-- 1 root root 3151584312 Jan  3 00:39 model.safetensors
-rw-r--r-- 1 root root       5094 Jan  3 00:38 README.md


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import PeftModel

base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

for param in base_model.parameters():
    param.requires_grad = False

full_model = PeftModel.from_pretrained(base_model, "/content/fine_tuned_model_v1.1")

# full_model = T5ForConditionalGeneration.from_pretrained("/content/fine_tuned_model")

trainable_params = [name for name, param in full_model.named_parameters() if param.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")
print("Trainable parameters:", trainable_params)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Number of trainable parameters: 0
Trainable parameters: []


In [ ]:
full_model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.3, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
       

In [ ]:
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer
import torch

# Configure the tokenizer's pad_token to be the eos_token
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token

#sample_input = "Write a SQL query to retrieve all customers who made purchases last year. TABLE CUSTOMER_PURCHASES 'contains customers and purchases' (CUSTOMER_NAME, PURCHASE_ID, PURCHASE_DATE)"
sample_input = "Write a SQL query to find all customers and the total amount they spent. TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID, CUSTOMER_NAME) TABLE PURCHASES 'contains purchase details' (PURCHASE_ID, CUSTOMER_ID, AMOUNT)"

inputs = tokenizer(sample_input, return_tensors="pt", truncation=True, padding=True)

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
full_model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output
with torch.no_grad():
    outputs = full_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,
        num_beams=4  # Use beam search for better results
    )

# Decode and print the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", decoded_output)


Generated Output: <pad> SELECT c.CUSTOMER_NAME, SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID; SELECT SUM(p.AMOUNT) AS TOTAL_PURCHASES FROM CUSTOMERS c LEFT JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_


In [ ]:
sample_input = "Write a SQL query to find the names of all customers who purchased a product in the 'Electronics' category. TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID, CUSTOMER_NAME) TABLE PURCHASES 'contains purchase details' (PURCHASE_ID, CUSTOMER_ID, PRODUCT_ID) TABLE PRODUCTS 'contains product details' (PRODUCT_ID, PRODUCT_NAME, CATEGORY)"

inputs = tokenizer(sample_input, return_tensors="pt", truncation=True, padding=True)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output
with torch.no_grad():
    outputs = full_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,
        num_beams=4  # Use beam search for better results
    )

# Decode and print the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", decoded_output)


Generated Output: <pad> SELECT c.CUSTOMER_NAME FROM CUSTOMERS c INNER JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID INNER JOIN PRODUCTS p ON p.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics'; SELECT c.CUSTOMER_NAME FROM CUSTOMERS c INNER JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID INNER JOIN PRODUCTS p ON p.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics'; SELECT c.CUSTOMER_NAME FROM CUSTOMERS c INNER JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID INNER JOIN PRODUCTS p ON p.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics'; SELECT c.CUSTOMER_NAME FROM CUSTOMERS c INNER JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID INNER JOIN PRODUCTS p ON p.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics'; SELECT c.CUSTOMER_NAME FROM CUSTOMERS c INNER JOIN PURCHASES p ON c.CUSTOMER_ID = p.CUSTOMER_ID INNER JOIN PRODUCTS p ON p.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics'; SELECT c.CUSTOMER_


In [ ]:
#sample_input = "Write a SQL query to find all orders and their customer names placed after January 1, 2023. TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID 'unique customer identifier', CUSTOMER_NAME 'name of the customer') TABLE ORDERS 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to customers', ORDER_DATE 'date the order was placed')"

sample_input = "What is kinjalization"
inputs = tokenizer(sample_input, return_tensors="pt", truncation=True, padding=True)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output
with torch.no_grad():
    outputs = full_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=1024,
        num_beams=2  # Use beam search for better results
    )

# Decode and print the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", decoded_output)


Generated Output: <pad> kinjalization is a term used to describe a process by which a group of people are grouped together in a particular way.


In [ ]:
sample_input = "Write a SQL query to find the total revenue generated in each region. TABLE REGIONS 'contains region details' (REGION_ID 'unique region identifier', REGION_NAME 'name of the region') TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID 'unique customer identifier', REGION_ID 'foreign key to regions') TABLE ORDERS 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to customers', TOTAL_AMOUNT 'total amount of the order')"

inputs = tokenizer(sample_input, return_tensors="pt", truncation=True, padding=True)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output
with torch.no_grad():
    outputs = full_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,
        num_beams=4  # Use beam search for better results
    )

# Decode and print the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", decoded_output)


Generated Output: <pad> SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM REGIONS r JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID GROUP BY r.REGION_NAME; SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM REGIONS r JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID GROUP BY r.REGION_NAME; SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM REGIONS r JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID GROUP BY r.REGION_NAME; SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM REGIONS r JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID GROUP BY r.REGION_NAME; SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM REGIONS r JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID; SELECT r.REG


In [ ]:
import nltk
nltk.download('punkt', force=True)
nltk.download('punkt_tab', force=True)
print(nltk.find('tokenizers/punkt'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/root/nltk_data/tokenizers/punkt


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer

def sliding_window_sentence_tokenize(tokenizer, text, max_length=512, stride=256):
    """
    Tokenizes text using a sliding window approach, ensuring sentence boundaries are respected.

    Parameters:
        tokenizer: The T5Tokenizer instance.
        text (str): Input text to be tokenized.
        max_length (int): Maximum token length for each chunk.
        stride (int): Overlap size (in tokens) between chunks.

    Returns:
        list: List of tokenized chunks (token IDs).
    """
    # Split text into sentences
    sentences = sent_tokenize(text)

    # Tokenize sentences and accumulate into chunks
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        tokenized_sentence = tokenizer(sentence, add_special_tokens=False)['input_ids']
        sentence_length = len(tokenized_sentence)

        # If adding the current sentence exceeds max_length, finalize the current chunk
        if current_length + sentence_length > max_length:
            chunks.append(current_chunk)
            current_chunk = tokenized_sentence[-stride:]  # Start new chunk with overlap
            current_length = len(current_chunk)
        else:
            current_chunk.extend(tokenized_sentence)
            current_length += sentence_length

    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [ ]:
# base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
full_model.to('cuda')

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.3, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
       

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import torch
max_length = 512
stride = 256

sample_input = "Using the tables and columns provided write a single SQL query to find the names of employees who worked on projects in the 'Technology' department. TABLE EMPLOYEES 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee') TABLE PROJECTS 'contains project details' (PROJECT_ID 'unique project identifier', DEPARTMENT_ID 'foreign key to departments') TABLE DEPARTMENTS 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department') TABLE EMPLOYEE_PROJECTS 'tracks employees assigned to projects' (EMPLOYEE_ID 'foreign key to employees', PROJECT_ID 'foreign key to projects')"

chunks = sliding_window_sentence_tokenize(tokenizer, sample_input, max_length, stride)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
# inputs = {key: value.to(device) for key, value in inputs.items()}
responses = []
for chunk in chunks:
  input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

# Generate output
  with torch.no_grad():
      outputs = full_model.generate(
          **input_chunk,
          max_length=256,
          #num_beams=4  # Use beam search for better results
          do_sample=True,  # Use sampling for more diverse outputs
          repetition_penalty=1.2,  # Penalize repeated tokens
          temperature=0.7,        # Add randomness
          top_k=10,               # Use top-k sampling
          top_p=0.9
      )
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  responses.append(response)

# Decode and print the output
#decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", responses)


Generated Output: ["<pad> SELECT e.EMPLOYEE_NAME FROM EMPLOYEES e INNER JOIN EMPLOYEE_PROJECTS ep ON e.EMPLOYEE_ID = ep.EMPLOYEE_ID INNER JOIN PROJECTS p ON ep.PROJECT_ID = p.PROJECT_ID WHERE p.DEPARTMENT_NAME = 'Technology'; SELECT e.EMPLOYEE_NAME FROM EMPLOYEE_PROJECTS ep INNER JOIN EMPLOYEES e ON ep.EMPLOYEE_ID = ep.EMPLOYEE_ID WHERE p.DEPARTMENT_NAME = 'Technology'; SELECT e.EMPLOYEE_NAME FROM EMPLOYEE_PROJECTS ep INNER JOIN EMPLOYEES e ON ep.EMPLOYEE_ID = e"]


In [ ]:
max_length = 512
stride = 256

sample_input = "Write a SQL query to find all products purchased by customers in the 'North' region. TABLE REGIONS 'contains region details' (REGION_ID 'unique region identifier', REGION_NAME 'name of the region') TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID 'unique customer identifier', REGION_ID 'foreign key to regions') TABLE ORDERS 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to customers') TABLE ORDER_DETAILS 'tracks products in orders' (ORDER_ID 'foreign key to orders', PRODUCT_ID 'foreign key to products') TABLE PRODUCTS 'contains product details' (PRODUCT_ID 'unique product identifier', PRODUCT_NAME 'name of the product')"

chunks = sliding_window_sentence_tokenize(tokenizer, sample_input, max_length, stride)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
# inputs = {key: value.to(device) for key, value in inputs.items()}
responses = []
for chunk in chunks:
  input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

# Generate output
  with torch.no_grad():
      outputs = full_model.generate(
          **input_chunk,
          max_length=512,
          num_beams=4  # Use beam search for better results
      )
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  responses.append(response)

# Decode and print the output
#decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", responses)


Generated Output: ["<pad> SELECT p.PRODUCT_NAME FROM REGIONS r INNER JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID INNER JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID INNER JOIN ORDER_DETAILS od ON o.PRODUCT_ID = od.PRODUCT_ID INNER JOIN PRODUCTS p ON od.PRODUCT_ID = p.PRODUCT_ID INNER JOIN ORDER_DETAILS odd ON od.PRODUCT_ID = odd.PRODUCT_ID INNER JOIN PRODUCTS p ON od.PRODUCT_ID = p.PRODUCT_ID WHERE r.REGION_NAME = 'North'; SELECT p.PRODUCT_NAME FROM REGIONS r INNER JOIN CUSTOMERS c ON r.REGION_ID = c.REGION_ID INNER JOIN ORDERS o ON c.CUSTOMER_ID = o.CUSTOMER_ID WHERE r.REGION_NAME = 'North'; SELECT p.PRODUCT_NAME FROM PRODUCTS p INNER JOIN ORDER_DETAILS od ON od.PRODUCT_ID = od.PRODUCT_ID WHERE od.ORDER_ID = od.ORDER_ID; SELECT p.PRODUCT_NAME FROM PRODUCTS p INNER JOIN ORDER_DETAILS od ON od.PRODUCT_ID = od.PRODUCT_ID WHERE od.ORDER_ID = od.ORDER_ID; SELECT p.PRODUCT_NAME FROM PRODUCTS p INNER JOIN ORDER_DETAILS od ON od.PRO"]


In [ ]:
max_length = 512
stride = 256

sample_input = "Write a SQL query to find the total sales for each salesperson in the 'Electronics' category. TABLE SALESPEOPLE 'contains salesperson details' (SALESPERSON_ID 'unique salesperson identifier', SALESPERSON_NAME 'name of the salesperson') TABLE ORDERS 'contains order details' (ORDER_ID 'unique order identifier', SALESPERSON_ID 'foreign key to salespeople') TABLE ORDER_DETAILS 'tracks products in orders' (ORDER_ID 'foreign key to orders', PRODUCT_ID 'foreign key to products', QUANTITY 'quantity of the product ordered') TABLE PRODUCTS 'contains product details' (PRODUCT_ID 'unique product identifier', CATEGORY 'category of the product', UNIT_PRICE 'price per unit of the product') TABLE CUSTOMERS 'contains customer details' (CUSTOMER_ID 'unique customer identifier') TABLE ORDER_CUSTOMERS 'tracks which customers placed which orders' (ORDER_ID 'foreign key to orders', CUSTOMER_ID 'foreign key to customers')"

chunks = sliding_window_sentence_tokenize(tokenizer, sample_input, max_length, stride)

# # Move inputs to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_model.to(device)
# inputs = {key: value.to(device) for key, value in inputs.items()}
responses = []
for chunk in chunks:
  input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

# Generate output
  with torch.no_grad():
      outputs = full_model.generate(
          **input_chunk,
          max_length=512,
          num_beams=4  # Use beam search for better results
      )
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  responses.append(response)

# Decode and print the output
#decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Output:", responses)


Generated Output: ["<pad> SELECT s.SALESPERSON_NAME, SUM(o.QUANTITY * p.UNIT_PRICE) AS TOTAL_SALE FROM SALESPEOPLE s INNER JOIN ORDERS o ON s.SALESPERSON_ID = o.SALESPERSON_ID INNER JOIN ORDER_DETAILS od ON o.ORDER_ID = od.ORDER_ID INNER JOIN PRODUCTS p ON od.PRODUCT_ID = p.PRODUCT_ID INNER JOIN CUSTOMERS c ON o.CUSTOMER_ID = c.CUSTOMER_ID INNER JOIN ORDER_CUSTOMERS oc ON c.CUSTOMER_ID = oc.CUSTOMER_ID WHERE p.CATEGORY = 'Electronics' GROUP BY s.SALESPERSON_NAME; SELECT s.SALESPERSON_NAME, SUM(o.QUANTITY * p.UNIT_PRICE) AS TOTAL_SALE FROM SALESPEOPLE s INNER JOIN ORDERS o ON s.SALESPERSON_ID = o.SALESPERSON_ID INNER JOIN ORDER_DETAILS od ON od.ORDER_ID = od.ORDER_ID INNER JOIN PRODUCTS p ON od.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Electronics' GROUP BY s.SALESPERSON_NAME; SELECT s.SALESPERSON_NAME, SUM(o.QUANTITY * p.UNIT_PRICE) AS TOTAL_SALE FROM SALESPEOPLE s INNER JOIN ORDERS o ON s.SALESPERSON_ID = o.SALESPERSON_ID INNER JOIN ORDER_DETAILS od ON od.ORDER_"]


In [ ]:
two_table = [
    {
        "question": "Write a SQL query to find the salary of each employee.",
        "schema": "TABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee') TABLE rad_salaries 'contains salary details' (EMPLOYEE_ID 'foreign key to rad_employees', SALARY 'monthly salary of the employee')"
    },
    {
        "question": "Write a SQL query to find the grades of students.",
        "schema": "TABLE rad_students 'contains student details' (STUDENT_ID 'unique student identifier', STUDENT_NAME 'name of the student') TABLE rad_grades 'contains grade details' (STUDENT_ID 'foreign key to rad_students', GRADE 'grade awarded')"
    },
    {
        "question": "Write a SQL query to find the authors of books.",
        "schema": "TABLE rad_books 'contains book details' (BOOK_ID 'unique book identifier', BOOK_TITLE 'title of the book') TABLE rad_authors 'contains author details' (AUTHOR_ID 'unique author identifier', BOOK_ID 'foreign key to rad_books', AUTHOR_NAME 'name of the author')"
    },
    {
        "question": "Write a SQL query to find the purchase date of orders.",
        "schema": "TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', PURCHASE_DATE 'date the order was placed') TABLE rad_order_items 'tracks items in orders' (ORDER_ID 'foreign key to rad_orders', ITEM_ID 'unique item identifier', QUANTITY 'quantity of the item purchased')"
    },
    {
        "question": "Write a SQL query to find customers and their email addresses.",
        "schema": "TABLE rad_customers 'contains customer details' (CUSTOMER_ID 'unique customer identifier', CUSTOMER_NAME 'name of the customer') TABLE rad_customer_contacts 'contains customer contact details' (CUSTOMER_ID 'foreign key to rad_customers', EMAIL 'email address of the customer')"
    }
]

# SELECT e.EMPLOYEE_NAME, SUM(s.SALARY) AS TOTAL_SALARY FROM rad_employees e INNER JOIN rad_salaries s ON e.EMPLOYEE_ID = s.EMPLOYEE_ID GROUP BY e.EMPLOYEE_NAME;
# SELECT s.STUDENT_NAME, g.GRADE FROM rad_students s LEFT JOIN rad_grades g ON s.STUDENT_ID = g.STUDENT_ID GROUP BY s.STUDENT_NAME, g.GRADE;
# SELECT b.BOOK_TITLE, a.AUTHOR_NAME, COUNT(a.AUTHOR_ID) AS TOTAL_AUTHORS FROM rad_books b LEFT JOIN rad_authors a ON b.BOOK_ID = a.BOOK_ID GROUP BY b.BOOK_TITLE, a.AUTHOR_NAME;
# SELECT o.ORDER_ID, o.PURCHASE_DATE, oi.QUANTITY FROM rad_orders o LEFT JOIN rad_order_items oi ON o.ORDER_ID = oi.ORDER_ID GROUP BY o.ORDER_ID, o.PURCHASE_DATE;
# SELECT c.CUSTOMER_NAME, cc.EMAIL FROM rad_customers c LEFT JOIN rad_customer_contacts cc ON c.CUSTOMER_ID = cc.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME, cc.EMAIL;



In [ ]:
import json

initial_text = "Using the given tables and columns details: "
end_text = ".Use different table aliases for different tables in given tables"
prompts = [f"{initial_text}{entry['question']}\n{entry['schema']}{end_text}" for entry in two_table]




In [ ]:
prompts

["Using the given tables and columns details: Write a SQL query to find the salary of each employee.\nTABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee') TABLE rad_salaries 'contains salary details' (EMPLOYEE_ID 'foreign key to rad_employees', SALARY 'monthly salary of the employee').Use different aliases for all tables",
 "Using the given tables and columns details: Write a SQL query to find the grades of all students.\nTABLE rad_students 'contains student details' (STUDENT_ID 'unique student identifier', STUDENT_NAME 'name of the student') TABLE rad_grades 'contains grade details' (STUDENT_ID 'foreign key to rad_students', GRADE 'grade awarded').Use different aliases for all tables",
 "Using the given tables and columns details: Write a SQL query to find the authors of all books.\nTABLE rad_books 'contains book details' (BOOK_ID 'unique book identifier', BOOK_TITLE 'title of the book') TABLE rad_authors 'con

In [ ]:
import re
for prompt in prompts:
  chunks = sliding_window_sentence_tokenize(tokenizer, prompt, max_length, stride)

  # # Move inputs to GPU if available
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # full_model.to(device)
  # inputs = {key: value.to(device) for key, value in inputs.items()}
  responses = []
  for chunk in chunks:
    input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

  # Generate output
    with torch.no_grad():
        outputs = full_model.generate(
            **input_chunk,
            max_length=512,
            num_beams=5  # Use beam search for better results
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  # Decode and print the output
  #decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print("sample_imput :", prompt)
  # print("Generated Output:", responses)
    if isinstance(responses, list):
      response_text = " ".join(responses)

  match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
  print(match.group(0).strip())

SELECT e.EMPLOYEE_NAME, SUM(s.SALARY) AS TOTAL_SALARY FROM rad_employees e INNER JOIN rad_salaries s ON e.EMPLOYEE_ID = s.EMPLOYEE_ID GROUP BY e.EMPLOYEE_NAME;
SELECT s.STUDENT_NAME, g.GRADE FROM rad_students s LEFT JOIN rad_grades g ON s.STUDENT_ID = g.STUDENT_ID GROUP BY s.STUDENT_NAME, g.GRADE;
SELECT b.BOOK_TITLE, a.AUTHOR_NAME, COUNT(a.AUTHOR_ID) AS TOTAL_AUTHORS FROM rad_books b LEFT JOIN rad_authors a ON b.BOOK_ID = a.BOOK_ID GROUP BY b.BOOK_TITLE, a.AUTHOR_NAME;
SELECT o.ORDER_ID, o.PURCHASE_DATE, oi.QUANTITY FROM rad_orders o LEFT JOIN rad_order_items oi ON o.ORDER_ID = oi.ORDER_ID GROUP BY o.ORDER_ID, o.PURCHASE_DATE;
SELECT c.CUSTOMER_NAME, cc.EMAIL FROM rad_customers c LEFT JOIN rad_customer_contacts cc ON c.CUSTOMER_ID = cc.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME, cc.EMAIL;


In [ ]:
prompts = ["Correct the following query - SELECT c.CUSTOMER_NAME, c.EMAIL FROM rad_customers c LEFT JOIN rad_contacts c ON c.CUSTOMER_ID = c.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME, c.EMAIL;"]

import re
for prompt in prompts:
  chunks = sliding_window_sentence_tokenize(tokenizer, prompt, max_length, stride)

  # # Move inputs to GPU if available
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # full_model.to(device)
  # inputs = {key: value.to(device) for key, value in inputs.items()}
  responses = []
  for chunk in chunks:
    input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

  # Generate output
    with torch.no_grad():
        outputs = full_model.generate(
            **input_chunk,
            max_length=512,
            num_beams=4  # Use beam search for better results
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  # Decode and print the output
  #decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print("sample_imput :", prompt)
  # print("Generated Output:", responses)
    if isinstance(responses, list):
      response_text = " ".join(responses)

  match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
  print(match.group(0).strip())

SELECT c.CUSTOMER_NAME, c.EMAIL FROM rad_customers c LEFT JOIN rad_contacts c ON c.CUSTOMER_ID = c.CUSTOMER_ID GROUP BY c.CUSTOMER_NAME, c.EMAIL;


In [ ]:
three_tables = [
    {
        "question": "Write a SQL query to find the total revenue generated in each region.",
        "schema": "TABLE rad_regions 'contains region details' (REGION_ID 'unique region identifier', REGION_NAME 'name of the region') TABLE rad_customers 'contains customer details' (CUSTOMER_ID 'unique customer identifier', REGION_ID 'foreign key to rad_regions') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to rad_customers', TOTAL_AMOUNT 'total amount of the order')"
    },
    {
        "question": "Write a SQL query to find the names of employees who worked on projects in the 'Technology' department.",
        "schema": "TABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee') TABLE rad_projects 'contains project details' (PROJECT_ID 'unique project identifier', DEPARTMENT_ID 'foreign key to rad_departments') TABLE rad_departments 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department')"
    },
    {
        "question": "Write a SQL query to find the most purchased product in each category.",
        "schema": "TABLE rad_products 'contains product details' (PRODUCT_ID 'unique product identifier', CATEGORY 'category of the product') TABLE rad_order_details 'tracks products in orders' (ORDER_ID 'foreign key to rad_orders', PRODUCT_ID 'foreign key to rad_products', QUANTITY 'quantity of the product ordered') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier')"
    },
    {
        "question": "Write a SQL query to find the total sales for each salesperson.",
        "schema": "TABLE rad_salespeople 'contains salesperson details' (SALESPERSON_ID 'unique salesperson identifier', SALESPERSON_NAME 'name of the salesperson') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', SALESPERSON_ID 'foreign key to rad_salespeople', TOTAL_AMOUNT 'total amount of the order') TABLE rad_order_details 'tracks products in orders' (ORDER_ID 'foreign key to rad_orders', PRODUCT_ID 'foreign key to rad_products', QUANTITY 'quantity of the product ordered')"
    },
    {
        "question": "Write a SQL query to list departments with more than 10 employees.",
        "schema": "TABLE rad_departments 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department') TABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', DEPARTMENT_ID 'foreign key to rad_departments') TABLE rad_salaries 'contains salary details' (EMPLOYEE_ID 'foreign key to rad_employees', SALARY 'monthly salary of the employee')"
    }
]


In [ ]:
import json

initial_text = "Using the given tables and columns details: "
end_text = ".Use different table names for different tables in given tables"
prompts = [f"{initial_text}{entry['question']}\n{entry['schema']}{end_text}" for entry in three_tables]



In [ ]:
for prompt in prompts:
  chunks = sliding_window_sentence_tokenize(tokenizer, prompt, max_length, stride)

  # # Move inputs to GPU if available
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # full_model.to(device)
  # inputs = {key: value.to(device) for key, value in inputs.items()}
  responses = []
  for chunk in chunks:
    input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

  # Generate output
    with torch.no_grad():
        outputs = full_model.generate(
            **input_chunk,
            max_length=512,
            num_beams=5  # Use beam search for better results
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  # Decode and print the output
  #decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if isinstance(responses, list):
    response_text = " ".join(responses)

  match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
  print(match.group(0).strip())

SELECT r.REGION_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_REVENUE FROM rad_regions r INNER JOIN rad_customers c ON r.REGION_ID = c.REGION_ID INNER JOIN rad_orders o ON c.CUSTOMER_ID = o.CUSTOMER_ID GROUP BY r.REGION_NAME;
SELECT e.EMPLOYEE_NAME FROM rad_employees e INNER JOIN rad_projects p ON e.EMPLOYEE_ID = p.EMPLOYEE_ID INNER JOIN rad_departments d ON p.DEPARTMENT_ID = d.DEPARTMENT_ID WHERE d.DEPARTMENT_NAME = 'Technology';
SELECT p.CATEGORY, MAX(o.QUANTITY) AS MAX_PURCHASED_PRODUCT FROM rad_products p LEFT JOIN rad_order_details od ON p.PRODUCT_ID = od.PRODUCT_ID LEFT JOIN rad_orders o ON od.ORDER_ID = o.ORDER_ID GROUP BY p.CATEGORY;
SELECT s.SALESPERSON_NAME, SUM(o.TOTAL_AMOUNT) AS TOTAL_SALE FROM rad_salespeople s INNER JOIN rad_orders o ON s.SALESPERSON_ID = o.SALESPERSON_ID INNER JOIN rad_order_details od ON o.ORDER_ID = od.ORDER_ID GROUP BY s.SALESPERSON_NAME;
SELECT d.DEPARTMENT_NAME FROM rad_departments d LEFT JOIN rad_employees e ON d.DEPARTMENT_ID = e.DEPARTMENT_ID LEFT JOIN rad_

In [ ]:
import re

if isinstance(responses, list):
  response_text = " ".join(responses)

match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
print(match.group(0).strip())

SELECT d.DEPARTMENT_NAME, COUNT(e.EMPLOYEE_ID) AS TOTAL_EMPLOYEES FROM rad_departments d LEFT JOIN rad_employees e ON d.DEPARTMENT_ID = e.DEPARTMENT_ID LEFT JOIN rad_salaries s ON e.EMPLOYEE_ID = s.EMPLOYEE_ID GROUP BY d.DEPARTMENT_NAME HAVING TOTAL_EMPLOYEES > 10;


In [ ]:
four_tables = [
    {
        "question": "Write a SQL query to find all customers who purchased electronics in the past year.",
        "schema": "TABLE rad_customers 'contains customer details' (CUSTOMER_ID 'unique customer identifier', CUSTOMER_NAME 'name of the customer') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to rad_customers', ORDER_DATE 'date the order was placed') TABLE rad_order_details 'tracks products in orders' (ORDER_ID 'foreign key to rad_orders', PRODUCT_ID 'foreign key to rad_products', QUANTITY 'quantity of the product ordered') TABLE rad_products 'contains product details' (PRODUCT_ID 'unique product identifier', CATEGORY 'category of the product', PRODUCT_NAME 'name of the product')"
    },
    {
        "question": "Write a SQL query to list all employees with salaries above $50,000 working in the 'HR' department.",
        "schema": "TABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee', DEPARTMENT_ID 'foreign key to rad_departments') TABLE rad_departments 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department') TABLE rad_salaries 'contains salary details' (EMPLOYEE_ID 'foreign key to rad_employees', SALARY 'monthly salary of the employee') TABLE rad_projects 'contains project details' (PROJECT_ID 'unique project identifier', DEPARTMENT_ID 'foreign key to rad_departments')"
    },
    {
        "question": "Write a SQL query to find the total revenue generated by each salesperson for 'Technology' products.",
        "schema": "TABLE rad_salespeople 'contains salesperson details' (SALESPERSON_ID 'unique salesperson identifier', SALESPERSON_NAME 'name of the salesperson') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', SALESPERSON_ID 'foreign key to rad_salespeople') TABLE rad_order_details 'tracks products in orders' (ORDER_ID 'foreign key to rad_orders', PRODUCT_ID 'foreign key to rad_products', QUANTITY 'quantity of the product ordered') TABLE rad_products 'contains product details' (PRODUCT_ID 'unique product identifier', CATEGORY 'category of the product', UNIT_PRICE 'price per unit of the product')"
    },
    {
        "question": "Write a SQL query to list all managers supervising more than 5 employees.",
        "schema": "TABLE rad_employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', EMPLOYEE_NAME 'name of the employee', MANAGER_ID 'foreign key to rad_employees') TABLE rad_departments 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department') TABLE rad_salaries 'contains salary details' (EMPLOYEE_ID 'foreign key to rad_employees', SALARY 'monthly salary of the employee') TABLE rad_projects 'contains project details' (PROJECT_ID 'unique project identifier', DEPARTMENT_ID 'foreign key to rad_departments')"
    },
    {
        "question": "Write a SQL query to find all regions with revenue exceeding $1,000,000.",
        "schema": "TABLE rad_regions 'contains region details' (REGION_ID 'unique region identifier', REGION_NAME 'name of the region') TABLE rad_customers 'contains customer details' (CUSTOMER_ID 'unique customer identifier', REGION_ID 'foreign key to rad_regions') TABLE rad_orders 'contains order details' (ORDER_ID 'unique order identifier', CUSTOMER_ID 'foreign key to rad_customers', TOTAL_AMOUNT 'total amount of the order') TABLE rad_order_details 'tracks products in orders' (ORDER_ID 'foreign key to rad_orders', PRODUCT_ID 'foreign key to rad_products')"
    }
]


In [ ]:
import json

initial_text = "Using the given tables and columns details and foreign key relations : "
end_text = ".Use different table aliases for all tables"
prompts = [f"{initial_text}{entry['question']}\n{entry['schema']}{end_text}" for entry in four_tables]


In [ ]:
for prompt in prompts:
  chunks = sliding_window_sentence_tokenize(tokenizer, prompt, max_length, stride)

  # # Move inputs to GPU if available
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # full_model.to(device)
  # inputs = {key: value.to(device) for key, value in inputs.items()}
  responses = []
  for chunk in chunks:
    input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

  # Generate output
    with torch.no_grad():
        outputs = full_model.generate(
            **input_chunk,
            max_length=512,
            num_beams=4  # Use beam search for better results
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  # Decode and print the output
  #decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  #print("Generated Output:", responses)
  if isinstance(responses, list):
    response_text = " ".join(responses)

  match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
  print(match.group(0).strip())

SELECT c.CUSTOMER_NAME FROM rad_customers c INNER JOIN rad_orders o ON c.CUSTOMER_ID = o.CUSTOMER_ID INNER JOIN rad_order_details od ON o.ORDER_ID = od.ORDER_ID INNER JOIN rad_products p ON od.PRODUCT_ID = p.PRODUCT_ID WHERE od.ORDER_DATE >= CURRENT_DATE - INTERVAL '1 year';
SELECT e.EMPLOYEE_NAME FROM rad_employees e INNER JOIN rad_departments d ON e.DEPARTMENT_ID = d.DEPARTMENT_ID INNER JOIN rad_salaries s ON e.EMPLOYEE_ID = s.EMPLOYEE_ID INNER JOIN rad_projects p ON d.DEPARTMENT_ID = p.DEPARTMENT_ID WHERE s.SALARY > 50000 AND d.DEPARTMENT_NAME = 'HR';
SELECT s.SALESPERSON_NAME, SUM(o.QUANTITY * p.UNIT_PRICE) AS TOTAL_REVENUE FROM rad_salespeople s INNER JOIN rad_orders o ON s.SALESPERSON_ID = o.SALESPERSON_ID INNER JOIN rad_order_details od ON o.ORDER_ID = od.ORDER_ID INNER JOIN rad_products p ON od.PRODUCT_ID = p.PRODUCT_ID WHERE p.CATEGORY = 'Technology' GROUP BY s.SALESPERSON_NAME;
SELECT m.MANAGER_ID FROM managers m INNER JOIN rad_employees e ON m.MANAGER_ID = e.MANAGER_ID INNER

In [ ]:
two_ten_samples = [
    {
        "question": "Write a SQL query to find all customers who made purchases over $100.",
        "schema": "TABLE customers 'contains customer details' (CUSTOMER_ID 'unique customer identifier', CUSTOMER_NAME 'name of the customer', EMAIL 'email address', PHONE 'phone number', ADDRESS 'home address', CITY 'city of residence', STATE 'state of residence', COUNTRY 'country of residence', ZIP_CODE 'postal code', REGISTRATION_DATE 'date of registration') TABLE purchases 'contains purchase details' (PURCHASE_ID 'unique purchase identifier', CUSTOMER_ID 'foreign key to customers', PRODUCT_ID 'product purchased', QUANTITY 'quantity purchased', UNIT_PRICE 'price per unit', TOTAL_AMOUNT 'total cost of the purchase', PURCHASE_DATE 'date of the purchase', PAYMENT_METHOD 'method of payment', STATUS 'status of the purchase', DISCOUNT 'discount applied')"
    },
    {
        "question": "Write a SQL query to retrieve the names of employees and their job titles who work in the 'IT' department.",
        "schema": "TABLE employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', FIRST_NAME 'first name of the employee', LAST_NAME 'last name of the employee', EMAIL 'email address', PHONE_NUMBER 'phone number', HIRE_DATE 'date of hire', JOB_ID 'foreign key to jobs', SALARY 'current salary', COMMISSION_PCT 'commission percentage', MANAGER_ID 'manager of the employee') TABLE jobs 'contains job details' (JOB_ID 'unique job identifier', JOB_TITLE 'title of the job', MIN_SALARY 'minimum salary for the job', MAX_SALARY 'maximum salary for the job', DEPARTMENT_ID 'foreign key to departments', LOCATION 'job location', JOB_DESCRIPTION 'description of the job', REQUIRED_SKILLS 'skills required for the job', EDUCATION_LEVEL 'required education level', EXPERIENCE 'years of experience required')"
    },
    {
        "question": "Write a SQL query to find all products in stock and their supplier details.",
        "schema": "TABLE products 'contains product details' (PRODUCT_ID 'unique product identifier', PRODUCT_NAME 'name of the product', CATEGORY 'category of the product', DESCRIPTION 'product description', UNIT_PRICE 'price per unit', QUANTITY_IN_STOCK 'quantity available in stock', REORDER_LEVEL 'minimum stock level for reorder', SUPPLIER_ID 'foreign key to suppliers', STATUS 'availability status', CREATED_AT 'date of product addition') TABLE suppliers 'contains supplier details' (SUPPLIER_ID 'unique supplier identifier', SUPPLIER_NAME 'name of the supplier', CONTACT_NAME 'contact person at supplier', PHONE 'phone number', EMAIL 'email address', ADDRESS 'business address', CITY 'city of operation', STATE 'state of operation', COUNTRY 'country of operation', POSTAL_CODE 'postal code')"
    },
    {
        "question": "Write a SQL query to retrieve the names of students enrolled in the 'Mathematics' course.",
        "schema": "TABLE students 'contains student details' (STUDENT_ID 'unique student identifier', FIRST_NAME 'first name of the student', LAST_NAME 'last name of the student', EMAIL 'email address', PHONE 'phone number', ENROLLMENT_DATE 'date of enrollment', PROGRAM 'academic program', YEAR 'year of study', MAJOR 'major subject', ADVISOR_ID 'academic advisor') TABLE courses 'contains course details' (COURSE_ID 'unique course identifier', COURSE_NAME 'name of the course', DEPARTMENT 'academic department offering the course', CREDITS 'number of credits for the course', INSTRUCTOR_ID 'foreign key to instructors', DESCRIPTION 'course description', PREREQUISITES 'required prerequisites for the course', SCHEDULE 'course schedule', LOCATION 'course location', MAX_ENROLLMENT 'maximum number of students allowed')"
    },
    {
        "question": "Write a SQL query to find all books borrowed in the last 6 months and their borrower details.",
        "schema": "TABLE books 'contains book details' (BOOK_ID 'unique book identifier', TITLE 'title of the book', AUTHOR 'author of the book', PUBLISHER 'publisher of the book', CATEGORY 'category of the book', LANGUAGE 'language of the book', ISBN 'ISBN number', PAGES 'number of pages', PRICE 'price of the book', AVAILABILITY 'availability status') TABLE borrowers 'contains borrower details' (BORROWER_ID 'unique borrower identifier', BORROWER_NAME 'name of the borrower', CONTACT_NUMBER 'contact number', EMAIL 'email address', MEMBERSHIP_DATE 'date of library membership', ADDRESS 'residential address', BORROW_DATE 'date the book was borrowed', RETURN_DATE 'expected return date', STATUS 'status of the borrowed book', FINE_AMOUNT 'fine amount if any')"
    },
    {
        "question": "Write a SQL query to list all vehicles serviced in the last month and their owners' contact details.",
        "schema": "TABLE vehicles 'contains vehicle details' (VEHICLE_ID 'unique vehicle identifier', MAKE 'manufacturer of the vehicle', MODEL 'model of the vehicle', YEAR 'year of manufacture', LICENSE_PLATE 'license plate number', OWNER_ID 'foreign key to owners', INSURANCE_STATUS 'insurance status', SERVICE_DATE 'date of last service', MILEAGE 'current mileage', STATUS 'current status of the vehicle') TABLE owners 'contains owner details' (OWNER_ID 'unique owner identifier', OWNER_NAME 'name of the vehicle owner', PHONE 'phone number', EMAIL 'email address', ADDRESS 'residential address', CITY 'city of residence', STATE 'state of residence', COUNTRY 'country of residence', ZIP_CODE 'postal code', REGISTRATION_DATE 'date of registration')"
    },
    {
        "question": "Write a SQL query to retrieve the details of all tickets issued for speeding violations.",
        "schema": "TABLE tickets 'contains ticket details' (TICKET_ID 'unique ticket identifier', VIOLATION_TYPE 'type of violation', VIOLATION_DATE 'date of the violation', VEHICLE_ID 'foreign key to vehicles', DRIVER_ID 'foreign key to drivers', FINE_AMOUNT 'amount of the fine', PAYMENT_STATUS 'status of the fine payment', ISSUING_OFFICER 'officer who issued the ticket', LOCATION 'location of the violation', DESCRIPTION 'additional details about the violation') TABLE drivers 'contains driver details' (DRIVER_ID 'unique driver identifier', DRIVER_NAME 'name of the driver', LICENSE_NUMBER 'driver's license number', PHONE 'contact phone number', EMAIL 'email address', ADDRESS 'residential address', CITY 'city of residence', STATE 'state of residence', COUNTRY 'country of residence', ZIP_CODE 'postal code', LICENSE_EXPIRY_DATE 'license expiration date')"
    },
    {
        "question": "Write a SQL query to find all patients who visited the hospital in the past week.",
        "schema": "TABLE patients 'contains patient details' (PATIENT_ID 'unique patient identifier', PATIENT_NAME 'name of the patient', AGE 'age of the patient', GENDER 'gender of the patient', PHONE 'contact phone number', EMAIL 'email address', ADDRESS 'residential address', CITY 'city of residence', STATE 'state of residence', ZIP_CODE 'postal code', DATE_OF_ADMISSION 'date of hospital admission') TABLE visits 'contains visit details' (VISIT_ID 'unique visit identifier', PATIENT_ID 'foreign key to patients', DOCTOR_ID 'foreign key to doctors', DATE_OF_VISIT 'date of the visit', REASON 'reason for the visit', PRESCRIPTION 'prescription details', DIAGNOSIS 'diagnosis details', FOLLOW_UP_DATE 'follow-up date if any', STATUS 'status of the visit', NOTES 'additional notes about the visit')"
    },
    {
        "question": "Write a SQL query to list all tenants whose rent is overdue.",
        "schema": "TABLE tenants 'contains tenant details' (TENANT_ID 'unique tenant identifier', NAME 'name of the tenant', PHONE 'contact phone number', EMAIL 'email address', LEASE_START_DATE 'start date of the lease', LEASE_END_DATE 'end date of the lease', UNIT_NUMBER 'apartment unit number', ADDRESS 'building address', CITY 'city of residence', STATE 'state of residence') TABLE rents 'contains rent payment details' (RENT_ID 'unique rent identifier', TENANT_ID 'foreign key to tenants', AMOUNT 'rent amount', DUE_DATE 'due date of the rent', PAYMENT_DATE 'date of rent payment', STATUS 'payment status', LATE_FEE 'late fee amount if any', PAYMENT_METHOD 'method of payment', TRANSACTION_ID 'transaction identifier', NOTES 'additional notes about the payment')"
    },
    {
        "question": "Write a SQL query to find all employees who were promoted in the last 2 years.",
        "schema": "TABLE employees 'contains employee details' (EMPLOYEE_ID 'unique employee identifier', FIRST_NAME 'first name of the employee', LAST_NAME 'last name of the employee', EMAIL 'email address', PHONE 'contact phone number', HIRE_DATE 'date of hire', DEPARTMENT_ID 'foreign key to departments', SALARY 'current salary', JOB_ID 'foreign key to jobs', STATUS 'employment status', PROMOTION_DATE 'date of last promotion') TABLE departments 'contains department details' (DEPARTMENT_ID 'unique department identifier', DEPARTMENT_NAME 'name of the department', LOCATION 'location of the department', MANAGER_ID 'manager of the department', PHONE 'department contact phone', EMAIL 'department email', BUDGET 'annual department budget', EMPLOYEE_COUNT 'number of employees in the department', CREATED_AT 'date the department was created', NOTES 'additional notes about the department')"
    }
]


In [ ]:
import json

initial_text = "Using the given tables and columns details and foreign key relations : "
end_text = ".Use different table aliases for all tables and use only tables provided"
prompts = [f"{initial_text}{entry['question']}\n{entry['schema']}{end_text}" for entry in two_ten_samples]


In [ ]:
for prompt in prompts:
  chunks = sliding_window_sentence_tokenize(tokenizer, prompt, max_length, stride)

  # # Move inputs to GPU if available
  # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # full_model.to(device)
  # inputs = {key: value.to(device) for key, value in inputs.items()}
  responses = []
  for chunk in chunks:
    input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

  # Generate output
    with torch.no_grad():
        outputs = full_model.generate(
            **input_chunk,
            max_length=512,
            num_beams=4  # Use beam search for better results
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

  # Decode and print the output
  #decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  #print("Generated Output:", responses)
  if isinstance(responses, list):
    response_text = " ".join(responses)

  match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)
  print(match.group(0).strip())

SELECT c.CUSTOMER_NAME FROM customers c LEFT JOIN purchases p ON c.CUSTOMER_ID = p.CUSTOMER_ID WHERE p.TOTAL_AMOUNT > 100;
SELECT e.EMPLOYEE_NAME, j.JOB_TITLE FROM employees e INNER JOIN jobs j ON e.JOB_ID = j.JOB_ID WHERE e.DEPARTMENT = 'IT';
SELECT p.PRODUCT_NAME, s.SUPPLIER_NAME, SUM(p.QUANTITY_IN_STOCK) AS TOTAL_STOCK FROM products p LEFT JOIN suppliers s ON p.SUPPLIER_ID = s.SUPPLIER_ID WHERE p.CREATED_AT >= CURRENT_DATE - INTERVAL '1 month' GROUP BY p.PRODUCT_NAME, s.SUPPLIER_NAME;
SELECT s.STUDENT_NAME, s.LAST_NAME FROM students s INNER JOIN courses c ON s.ENROLLMENT_DATE = c.COURSE_ID WHERE c.COURSE_NAME = 'Mathematics' GROUP BY s.STUDENT_NAME, s.LAST_NAME;
SELECT b.BORROWER_NAME, b.TITLE, b.AUTHOR, b.PUBLISHER, b.CATEGORY, b.LANGUAGE, b.PRICE, b.AVAILABILITY FROM books b LEFT JOIN borrowers b ON b.BORROWER_ID = b.BORROWER_ID WHERE b.BORROW_DATE >= CURRENT_DATE - INTERVAL '6 MONTHS' AND b.AVAILABILITY >= CURRENT_DATE;
SELECT v.VEHICLE_NAME, o.OWNER_NAME, o.PHONE, o.EMAIL, o.ADD